youtube video to text summarizer

In [ ]:
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi

# Load video transcript in chunks
video_id =  "SdpSgBj4Fog"
transcript = YouTubeTranscriptApi.get_transcript(video_id)

# Initialize summarization model
summarizer = pipeline("summarization")

# Process transcript in chunks
chunk_size = 500  # Adjust as needed
for i in range(0, len(transcript), chunk_size):
    chunk = transcript[i:i + chunk_size]
    transcript_text = " ".join([entry["text"] for entry in chunk])
    summary = summarizer(transcript_text, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, temperature=0.5)
    print(f"Summary for chunk {i}-{i + chunk_size}:")
    print(summary[0]["summary_text"])


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Summary for chunk 0-500:
 Today let us learn about long and short duration . Let's look at another example andy's school remained closed for one week for christmas but his goal remained closed  for 4 weeks for summer vacation . It means the school remained . closed for a short duration or shorter time for a long duration or longer time for summer .


youtube video to summarized audio


In [ ]:
!pip install gtts
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi
from gtts import gTTS
import IPython.display as ipd

# Load video transcript
video_id = "SdpSgBj4Fog"
transcript = YouTubeTranscriptApi.get_transcript(video_id)

# Initialize summarization model
summarizer = pipeline("summarization")

# Process transcript and generate summary
transcript_text = " ".join([entry["text"] for entry in transcript])
summary = summarizer(transcript_text, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, temperature=0.5)



# Display and save audio summary
audio_summary = summary[0]["summary_text"]
tts = gTTS(audio_summary, lang='en')
tts.save("audio_summary.mp3")

# Display audio
ipd.Audio("audio_summary.mp3")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


back ground remover


In [ ]:
 import torch
from torchvision import models, transforms
from PIL import Image
import os

# Load the pre-trained DeepLabV3 model
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load and preprocess the image
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0)
    return input_tensor

# Perform semantic segmentation to get the mask
def segment_image(image_tensor, model):
    with torch.no_grad():
        output = model(image_tensor)['out'][0]
        predictions = output.argmax(0)
    return predictions

# Remove the background using the predicted mask
def remove_background(input_image_path, output_image_path):
    input_tensor = load_and_preprocess_image(input_image_path)
    mask = segment_image(input_tensor, model)

    # Create a binary mask
    binary_mask = (mask == 15).float()

    # Resize the binary mask to match the input image dimensions
    binary_mask = transforms.Resize(input_tensor.shape[2:])(transforms.ToPILImage()(binary_mask))

    # Convert binary_mask to a NumPy array with the correct shape
    binary_mask = transforms.ToTensor()(binary_mask).permute(1, 2, 0).numpy()

    # Apply the mask to the original image
    result_image = input_tensor.squeeze(0).permute(1, 2, 0).numpy() * binary_mask

    # Save the result
    result_image = Image.fromarray((result_image * 255).astype('uint8'))
    result_image.save(output_image_path)

# Example usage
input_image_path = "/content/White formal.jpg"

# Get the directory and filename from the input path
input_directory, input_filename = os.path.split(input_image_path)

# Create the output image path by modifying the filename
output_image_filename = f"output_{input_filename}"
output_image_path = os.path.join(input_directory, output_image_filename)

remove_background(input_image_path, output_image_path)


In [ ]:

import torch
from torchvision import models, transforms
from PIL import Image
import os
import gradio as gr

# Load the pre-trained DeepLabV3 model
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load and preprocess the image
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0)
    return input_tensor

# Perform semantic segmentation to get the mask
def segment_image(image_tensor, model):
    with torch.no_grad():
        output = model(image_tensor)['out'][0]
        predictions = output.argmax(0)
    return predictions

# Remove the background using the predicted mask
def remove_background(input_image_path):
    input_tensor = load_and_preprocess_image(input_image_path)
    mask = segment_image(input_tensor, model)

    # Create a binary mask
    binary_mask = (mask == 15).float()

    # Resize the binary mask to match the input image dimensions
    binary_mask = transforms.Resize(input_tensor.shape[2:])(transforms.ToPILImage()(binary_mask))

    # Convert binary_mask to a NumPy array with the correct shape
    binary_mask = transforms.ToTensor()(binary_mask).permute(1, 2, 0).numpy()

    # Apply the mask to the original image
    result_image = input_tensor.squeeze(0).permute(1, 2, 0).numpy() * binary_mask

    # Convert the result to a PIL Image
    result_image = Image.fromarray((result_image * 255).astype('uint8'))
    return result_image

# Gradio Interface
iface = gr.Interface(fn=remove_background, inputs="image", outputs="image", live=True)

# Launch the Gradio interface
iface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e588b581fc0f9c1e51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer

# Load Blenderbot model and tokenizer
model_name = "facebook/blenderbot-3B"
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)

# Function to chat with the bot
def chat_with_bot(user_input, model, tokenizer, max_length=50):
    input_ids = tokenizer.encode(user_input, return_tensors="pt", max_length=1024)
    output = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Interactive chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = chat_with_bot(user_input, model, tokenizer)
    print("Bot:", response)
